# How to automate IP ranges calculations in Azure using PowerShell

> The notebook does work on Linux and [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/eosfor/scripting-notes/HEAD)

## Scenario

Assume we got the IP rage of `10.172.0.0/16` from the network team for planned Azure Landing Zones. We want to automate this by making a tool, which will automatically calculate IP ranges for us given some high level and simple to understand details about the future networks.

This notebook shows how to do it using the [ipmgmt](https://github.com/eosfor/ipmgmt) module. So let us go and install it first

In [ ]:
Install-Module ipmgmt -Scope CurrentUser

And import it

In [ ]:
Import-Module ipmgmt

The module contains only two cmdlets. The `Get-VLSMBreakdown` breaks down a range into smaller ones, meaning we can use it to break a range into VNETs and then each VNET into subnets. The `Get-IPRanges` cmdlet given the list of ranges in-use and the "root" range tries to find a free slot of the specified size. It can be used to avoid losing IP space

In [ ]:
Get-Command -Module ipmgmt

Let us look at how we can break down our big "root" IP range into smaller ones. For that, we just need to prepare a list of smaller sub-ranges in the form of PowerShell hashtables, like so `@{type = "VNET-HUB"; size = (256-2)}`. Here we say that the name of the range is `VNET-HUB` and the size is simply `256-2`, which is the maximum number of IPs in the `/24` subnet minus 2, the first and the last.

If we need more than one, we just make an array of these hashtables

In [ ]:
$subnets = @{type = "VNET-HUB"; size = (256-2)},
           @{type = "VNET-A"; size = (256-2)}

Now everything is ready and we can try to break the "root" network

In [ ]:
Get-VLSMBreakdown -Network 10.172.0.0/16 -SubnetSize $subnets | ft type, network, netmask, *usable, cidr -AutoSize

Here we got two ranges named `VNET-A` and `VNET-HUB`. However, by doing so we made a few unused slots in the `root` range. They are marked as `reserved`, just for our convenience. It shows what happens to the range when we break it down. The smaller sub-ranges you make, the more of such unused ranges you get in the end.

Another way of doing this is by using CIDR notation, like this

In [ ]:
$subnets = @{type = "GTWSUBNET"; cidr = 27},
@{type = "DMZSUBNET"; cidr = 26},
@{type = "EDGSUBNET"; cidr = 27},
@{type = "APPSUBNET"; cidr = 26},
@{type = "CRESUBNET"; cidr = 26}

Get-VLSMBreakdown -Network 10.10.5.0/24 -SubnetSizeCidr $subnets | ft -AutoSize

Ok, let's try to use what we've got. For that, we need to authenticate to Azure. When running locally you can just do

In [ ]:
Login-AzAccount

In Binder however, it needs to be a bit different, like this

In [ ]:
Connect-AzAccount -UseDeviceAuthentication

Lets first create a resource group to work with

In [ ]:
New-AzResourceGroup -ResourceGroupName 'vnet-test' -Location 'eastus2' | Out-Null

Once authenticated, we can create networks, for example, like this. Here we first filter out the `reserved` ones for simplicity.

In [ ]:
$rootIprange = '10.172.0.0/16'
$subnets = @{type = "VNET-HUB"; size = (256-2)},
           @{type = "VNET-A"; size = (256-2)}

$vnets = Get-VLSMBreakdown -Network $rootIprange -SubnetSize $subnets | ? type -ne 'reserved'

$vnets | % {
    New-AzVirtualNetwork -Name  $_.type -ResourceGroupName 'vnet-test' `
                         -Location 'eastus2' -AddressPrefix "$($_.Network)/$($_.cidr)" | 
                         select name, @{l='AddressSpace'; e={$_.AddressSpace.AddressPrefixes}}, ResourceGroupName, Location
}

Ok, assume, at some point, we need to add a few more networks. And at the same time we may want to reuse one of those `reserved` slots, if it matches the size. This is what `Get-IPRanges` does. It takes a list of IP ranges "in-use" and returns slots that can fit the range in question. For example, in our case, we have a "base range" of `10.10.0.0/16` and two ranges in-use `10.10.5.0/24`, `10.10.7.0/24`. We are looking for a range of the size `/22`. So the cmdlet recommends us to use the `10.172.4.0/22`, which is one of the `reserved` ranges from the previous example

In [ ]:
Get-IPRanges -Networks "10.172.1.0/24", "10.172.0.0/24" -CIDR 22 -BaseNet "10.172.0.0/16" | ft -AutoSize

What if we need to find more than just one range at a time? Need not worry. We can do it with this simple script. And we are using Azure as a source of truth, as we can always query it for the real IP ranges, which are in use.

So what we need to do is relatively simple:

- make a list of sizes we want to create and put it into a variable - `$cidrRange`
- pull the ranges from Azure. We assume they are in use by someone - `$existingRanges`
- cast whatever we pulled from azure the `System.Net.IPNetwork` for correctness. This type is used inside the `ipmgmt` module to store information about networks and do all the calculations, comparisons, etc.
- now we run through the list of sizes, for each of them ask `Get-IPRanges` to find a proper slot, and accumulate the results

Now we just need to mark the new ranges as `free`, to see what we've got. For that,  we compare what we have in Azure to what we just calculated and mark the difference accordingly

In [ ]:
$cidrRange = 25,25,24,24,24,24,23,25,26,26 | sort
$existingRanges = (Get-AzVirtualNetwork -ResourceGroupName vnet-test | 
    select name, @{l = "AddressSpace"; e = { $_.AddressSpace.AddressPrefixes }}, ResourceGroupName, Location |
    select -expand AddressSpace)
$existingNetworks = $existingRanges | % {[System.Net.IPNetwork]$_}
$nets = $existingRanges

$ret = @()

$cidrRange | % {
    $ret = Get-IPRanges -Networks $nets -CIDR $_ -BaseNet "10.172.0.0/16"
    $nets = ($ret | select @{l="range"; e = {"$($_.network)/$($_.cidr)"}}).range
}

$ret | % {
    if ( -not ($_ -in $existingNetworks)) {$_.IsFree = $true}
}

$ret | ft -AutoSize

And this gives us all the necessary information to add a few more networks.

## Cleanup

In [ ]:
Remove-AzResourceGroup -ResourceGroupName 'vnet-test' -Force | Out-Null